In [1]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes

# Generacion de clave privada
private_key = ec.generate_private_key(ec.SECP256K1())
public_key = private_key.public_key()

# Firma de mensaje
message = b"Transaccion blockchain"
signature = private_key.sign(message, ec.ECDSA(hashes.SHA256()))

# Verificacion
try:
    public_key.verify(signature, message, ec.ECDSA(hashes.SHA256()))
    print("Firma valida")
except:
    print("Firma invalida")

Firma valida


In [2]:
import hashlib
import json
from datetime import datetime
import random

class Block:
    def __init__(self, index, transaction, previous_hash):
        self.index = index
        self.timestamp = str(datetime.now())
        self.transaction = transaction  # Solo una transacción por bloque
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.compute_hash()
        self.stakeholders = {}  # Para PoS
        self.authorized_validators = []  # Para PoA

    def compute_hash(self):
        block_data = {
            'index': self.index,
            'timestamp': self.timestamp,
            'transaction': self.transaction,
            'previous_hash': self.previous_hash,
            'nonce': self.nonce
        }
        block_string = json.dumps(block_data, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

    def pow(self, difficulty=4):
        while self.hash[:difficulty] != '0' * difficulty:
            self.nonce += 1
            self.hash = self.compute_hash()
        print(f"Bloque minado con PoW: {self.hash}")

    def pos(self):
        # Simplificación: elegir validador al azar según stake
        if not self.stakeholders:
            print("No hay participantes con stake para validar.")
            return

        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for validator, stake in self.stakeholders.items():
            current += stake
            if current >= pick:
                print(f"Bloque validado con PoS por: {validator}")
                break

    def poa(self, proposed_validator=None):
        if not self.authorized_validators:
            print("No hay validadores autorizados definidos.")
            return

        if proposed_validator:
            if proposed_validator in self.authorized_validators:
                print(f"Bloque validado con PoA por: {proposed_validator}")
            else:
                print(f"ERROR: {proposed_validator} no está autorizado para validar este bloque.")
        else:
            validator = random.choice(self.authorized_validators)
            print(f"Bloque validado con PoA por: {validator}")

In [3]:
def create_genesis_block():
    return Block(0, "Primer Block", "0")

def create_new_block(previous_block, transaction):
    index = previous_block.index + 1
    previous_hash = previous_block.hash
    new_block = Block(index, transaction, previous_hash)
    return new_block

blockchain = []

# Crear el bloque génesis
genesis_block = create_genesis_block()
blockchain.append(genesis_block)

# Crear un nuevo bloque
new_block = create_new_block(genesis_block, "Transaccion 1")
blockchain.append(new_block)

# Crear otro nuevo bloque
new_block = create_new_block(blockchain[-1], "Transaccion 2")
blockchain.append(new_block)

# Imprimir la cadena de bloques
for block in blockchain:
    print(f"Bloque {block.index}:")
    print(f"  Timestamp: {block.timestamp}")
    print(f"  Transaccion: {block.transaction}")
    print(f"  Hash anterior: {block.previous_hash}")
    print(f"  Hash: {block.hash}")
    print()

Bloque 0:
  Timestamp: 2025-05-15 11:32:03.874030
  Transaccion: Primer Block
  Hash anterior: 0
  Hash: 08b0360f1bea2294ecdf7c2c8df8af276b9a11867b540993ec0385341c48123c

Bloque 1:
  Timestamp: 2025-05-15 11:32:03.874030
  Transaccion: Transaccion 1
  Hash anterior: 08b0360f1bea2294ecdf7c2c8df8af276b9a11867b540993ec0385341c48123c
  Hash: 4cfe3eb7e6c7cd88832ce118b722a718c605363e737699ddae9cf67356018426

Bloque 2:
  Timestamp: 2025-05-15 11:32:03.874030
  Transaccion: Transaccion 2
  Hash anterior: 4cfe3eb7e6c7cd88832ce118b722a718c605363e737699ddae9cf67356018426
  Hash: 04ab5dbf0f866963dac315a3d7afd5d668093bb1b4b31c757d23632cf208df44



In [4]:
# Ejemplo de uso PoW

genesis_block = Block(0, [], "0")
genesis_block.pow(difficulty = 5)

Bloque minado con PoW: 00000566f16e5d99a6437be4e3bb9ac42319755c6c4a69bfd42a23a03d240d8d


In [5]:
#Ejemplo de uso PoS

genesis_block = Block(0, [], "0")
genesis_block.stakeholders = {
    "Alice": 10,
    "Bob": 80,
    "Charlie": 10
}

genesis_block.pos()

Bloque validado con PoS por: Bob


In [6]:
# Ejemplo de uso PoA

genesis_block = Block(0, [], "0")
genesis_block.authorized_validators = ["Alice", "Bob", "Charlie"]

genesis_block.poa("Alice")  
genesis_block.poa("Oliver") # mostrará error porque no está autorizado
genesis_block.poa()         # aleatorio

Bloque validado con PoA por: Alice
ERROR: Oliver no está autorizado para validar este bloque.
Bloque validado con PoA por: Alice
